In [17]:
import warnings
warnings.simplefilter('ignore')

In [1]:
from urllib import request as req

data_dir = '/Users/ssv/newprolab/project01'
filename = 'gender_age_dataset.txt'
file_path = '/'.join([data_dir, filename])
url = 'http://data.newprolab.com/data-newprolab-com/project01/' + filename

# req.urlretrieve(url, file_path)

In [13]:
import pandas as pd
import json
from urllib.parse import urlparse
from datetime import datetime
from tqdm import tqdm_notebook
import pickle

In [3]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [4]:
df = pd.read_csv(file_path, sep='\t')

In [6]:
df['user_json'] = df['user_json'].apply(lambda x: json.loads(x))

In [7]:
df['url_counts'] = df['user_json'].apply(lambda x: len(x['visits']))

In [8]:
#  Формирование столбца, содержащего массив распарсенных урлов, 
#  тип ParseResult(scheme='http', netloc='zebra-zoya.ru', 
#  path='/200028-chehol-organayzer-dlja-macbook-11-grid-it.html', 
#  params='', query='utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun', fragment='')

df['url_list'] = df['user_json'].apply(lambda x: [urlparse(x['url']) for x in x['visits']])

In [9]:
#  Формирование столбца, содержащего массив дат, полученных из timestamp, 
#  тип datetime.datetime(2015, 3, 24, 10, 25, 44)
df['ts_list'] = df['user_json'].apply(lambda x: \
                                       [datetime.fromtimestamp(int(float(str(x['timestamp'])[:10] + \
                                                            '.' + str(x['timestamp'])[10:13]))) \
                                        for x in x['visits']])

In [10]:
# Выбор уникальных урлов из массива распарсенных урлов, ParseResult.netloc (str)
df['url_netloc'] = df['url_list'].apply(lambda x: list(set([y.scheme + '://' + y.netloc for y in x ])))

In [13]:
#========================================================

In [11]:
# Перекодировка урлов x.encode('idna').decode('idna') с очисткой от мусора 

i = 0
for items in tqdm_notebook(df['url_netloc']):    
    for netloc in items:
        try:
            netloc.encode('idna').decode('idna')
        except:           
            df['url_netloc'][i].remove(netloc)
            # print('netloc -- {} ==>'.format(i), netloc)
            netloc = ''
            continue     
    i += 1

A Jupyter Widget

In [46]:
import pickle
with open('url_for_scraping.pickle', 'wb') as file:
    pickle.dump(url_for_scraping, file)

In [ ]:
# with open('url_for_scraping.pickle', 'rb') as file:
#     url_for_scraping = pickle.load(file)

In [28]:
df_med_keywords = pd.DataFrame()
df_med_keywords[['uid', 'url_netloc']] = df[['uid', 'url_netloc']]
df_med_keywords['raw_keywords'] = ''

In [29]:
df_med_keywords.columns

Index(['uid', 'url_netloc', 'raw_keywords'], dtype='object')

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

i = 0
for index, row in tqdm_notebook(df_med_keywords.iterrows()):
    keywords = []
    tag = ''
    for url in row['url_netloc']:
        try:
            html = BeautifulSoup(urlopen(url).read())        
            tag = html.find('meta', {'name':'description'})    
            if tag != None:
                keywords.append(tag.attrs['content'])
            tag = html.find('meta', {'name':'keywords'})    
            if tag != None:
                keywords.append(tag.attrs['content'])    
        except:
            continue
            
    df_med_keywords.loc[index]['raw_keywords'] = keywords

    i += 1
    if i == 1000 or index == 41137:
        i = 0
        filename = 'keywords_' + str(index) + '.pickle'
        with open(filename, 'wb') as file:
            pickle.dump(df_med_keywords, file)
            
with open('df_med_keywords.pickle', 'wb') as file:
    pickle.dump('df_med_keywords.pickle', file)            

In [47]:
print(df_med_keywords.loc[1]['raw_keywords'])



['Новости информационного агентства Интерфакс: Нападение на ведущую Эха Москвы, Референдум в Каталонии, Дело Алексея Улюкаева, Крах ВИМ-Авиа, Дипломатическая война с США', 'EuroAvtoprokat.Ru предлагает лучшие цены на прокат автомобилей в Европе, аренда автомобилей со скидками во всех крупнейших городах и аэропортах Испании, Италии, Германии, Швейцарии, Швеции и др.', 'автопрокат, прокат автомобилей, аренда автомобилей, прокат машин, бронирование авто, авто на прокат, испании, италии, швейцарии, германии, sixt, hertz, budget, avis, europcar', 'car rental', 'Android в России: Новости, советы, помощь', 'android телефон планшет не работает помощь проблема', 'Ответы на сканворды. Значение слова Ответы на сканворды. Ответы на кроссворды и сканворды. Поиск слов по маске. Слова из 5 букв.', 'слова 5 из букв, поиск по маске, Ответы на сканворды, подобрать слово, разгадать слово, разгадать сканворд, кроссворд', 'This is a user-to-user discussion forum', 'forum,bbs,discussion,bulletin board,Net T

In [42]:
df_med_keywords.loc[0]['raw_keywords']

['Zebra Zoya – магазин креативных подарков и стильных товаров для дома. Только качественные товары от лучших мировых брендов. А именно: подарки, товары для кухни и быта, косметика, товары для детей и многое другое.',
 'Зебра Зоя,интернет магазин,интернет гипермаркет,товары для дома,товары для кухни,подарки,Магазин подарков,магазин товаров для дома',
 'Последние новости в Яндекс.Новостях: Актуальные новости России, регионов и областей, важные и свежие новости дня.',
 '']

In [38]:
df_med_keywords.loc[1]['raw_keywords']

['gghghghg', 'hghghghgh']

In [57]:
print(df.loc[1725]['url_netloc'])

['http://www.babla.ru', 'http://rdmservice.ru', 'http://go.youlamedia.com', 'http://cekctube.net', 'http://www.mamba.ru', 'http://helpanu.ru', 'http://www.wordreference.com', 'http://www.1zoom.ru', 'https://www.google.com', 'http://perdos.ru', 'http://www.androidlost.com', 'http://spinet.ru', 'http://otzovik.com', 'http://www.bolshoyvopros.ru', 'http://www.intellicast.com', 'http://uhhan.ru', 'http://www.calend.ru', 'http://ymka.tv', 'http://wikibit.me', 'https://topface.com', 'http://yamagu.ru', 'http://zdanie.info', 'http://topface.com', 'http://calculatori.ru', 'http://operatorgid.ru']
